In [3]:
import sys
import os
import json
import datetime
import numpy as np
import pandas as pd

In [4]:
params={'startdate':'2018-05-10','year':15,'size':99.5,'weight':1.5,'averagetime':3.4,"scenario":"lm_model","type":"month"
        ,"plant":"solar", "contruction":150000000,"investment":180000000,"othercost":25000000,"debt":120000000,"interest":0.05,
        "unredeemed":12321321123,"duration":12,"repayment_method":"cpm","repayment_term":"y"}

In [5]:
def toQuarter(data):
    quarter= []
    for i in range(0,data.index.size):
        quarter.append(str(data.index.year[i])+'-'+str(data.index.quarter[i]))
    return quarter

def date_calc(startdate,debt,duration,repayment_term):
    result=pd.DataFrame(index=pd.period_range(start=startdate,periods=duration*12+1,freq="m"))
    result['days']=result.index.day
    start=pd.to_datetime(startdate, format='%Y-%m-%d', errors='ignore')
    startday=start.date().timetuple()
    result.days[0]=result.days[0]-startday.tm_mday
    result.days[result.shape[0]-1]=startday.tm_mday
    if repayment_term=="m":
        return result
    elif repayment_term=="q":
        result.index=toQuarter(result)
        result=result.groupby(result.index).sum()
        return result
    elif repayment_term=="y":
        result.index=result.index.year
        result=result.groupby(result.index).sum()
        return result

In [6]:
date_calc(params["startdate"],params["debt"],params["duration"],params["repayment_term"])

,days
2018,235
2019,365
2020,366
2021,365
2022,365
2023,365
2024,366
2025,365
2026,365
2027,365


In [307]:
def CAM_calc(data,debt,interest,type,year,duration):
    data["debt"]=0; data["remained_debt"]=0;data["interest"]=0;
    if(type=="m"):
        for i in range(1,duration+1):
            data.debt[12*i]=debt/duration;
        for j in range(0, duration):
            data.remained_debt[12*j]=debt-debt/duration*j
            for k in range(0,12):
                    data.interest[12*j+k]=data.remained_debt[12*j]*interest/12
        return data
    if(type=="q"):
        for i in range(1, duration+1):
            data.debt[4*i]=debt/duration;
        for j in range(0, duration):
            data.remained_debt[4*j]=debt-debt/duration*j
            for k in range(0,4):
                    data.interest[4*j+k]=data.remained_debt[4*j]*interest/4
        return data
    if(type=="y"):
        for i in range(0, duration+1):
            data.debt.iloc[i]=debt/duration
            data.remained_debt.iloc[i]=debt-debt/duration*i
            data.interest.iloc[i]=data.remained_debt.iloc[i]*interest
        return data
        

In [308]:
test
CAM_calc(test,params['debt'],params['interest'],params["repayment_term"],params["year"],params["duration"])

/Users/youngji/.local/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,days,debt,remained_debt,interest
2018,235,10000000.0,120000000.0,6000000.0
2019,365,10000000.0,110000000.0,5500000.0
2020,366,10000000.0,100000000.0,5000000.0
2021,365,10000000.0,90000000.0,4500000.0
2022,365,10000000.0,80000000.0,4000000.0
2023,365,10000000.0,70000000.0,3500000.0
2024,366,10000000.0,60000000.0,3000000.0
2025,365,10000000.0,50000000.0,2500000.0
2026,365,10000000.0,40000000.0,2000000.0
2027,365,10000000.0,30000000.0,1500000.0


In [296]:
test.debt

2018    100
2019    100
2020    100
2021    100
2022    100
2023    100
2024    100
2025    100
2026    100
2027    100
2028    100
2029    100
2030    100
Name: debt, dtype: int64

In [271]:
range(1, 12)

range(1, 12)

In [192]:
def CPM_calc(data,debt,interest,type,year):
    if(type=="m"):
        term=year*12
        divided_interest=(1+interest/12)
        money=(debt*divided_interest*(1+divided_interest)**term)/((1+divided_interest)**term-1)
        data["payback"]=money;
        data.payback[0]=0;
        return data
    elif(type=="q"):
        term=year*4
        divided_interest=(1+interest/4)
        money=(debt*divided_interest*(1+divided_interest)**term)/((1+divided_interest)**term-1)
        data["payback"]=money
        data.payback[0]=0;
        return data
    elif(type=="y"):
        term=year
        divided_interest=(1+interest)
        money=(debt*divided_interest*(1+divided_interest)**term)/((1+divided_interest)**term-1)
        data["payback"]=money
        data.payback[0]=0;
        return data
    else:
        return "error"

def CAM_calc(data,debt,interest,type,year):
    if(type=="m"):
    
    if(type=="q"):
term=year*4
divided_interest=(1+interest/4)
money=(debt*divided_interest*(1+divided_interest)**term)/((1+divided_interest)**term-1)
data["payback"]=money
data.payback[0]=0;
        return data
    if(type=="y"):
        term=year
        divided_interest=(1+interest)
        money=(debt*divided_interest*(1+divided_interest)**term)/((1+divided_interest)**term-1)
        data["payback"]=money
        data.payback[0]=0;
        return data

IndentationError: expected an indented block (<ipython-input-192-1b70fdd4cd67>, line 29)

In [35]:
result=CPM_calc(test,params['debt'],params['interest'],params["repayment_term"],params["year"])
result

/Users/youngji/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,days,payback
2018-05,21,0.0
2018-06,30,120500000.0
2018-07,31,120500000.0
2018-08,31,120500000.0
2018-09,30,120500000.0
2018-10,31,120500000.0
2018-11,30,120500000.0
2018-12,31,120500000.0
2019-01,31,120500000.0
2019-02,28,120500000.0


In [20]:
index_quarter
index_month=pd.period_range(start=params["startdate"],periods=params["year"]*12+1,freq="m")
index_quarter=pd.period_range(start=params["startdate"],periods=params["year"]*4+1,freq="q")
index_year=pd.period_range(start=params["startdate"],periods=params["year"]+1,freq="y")

pred['rec_price']=100;pred['size']=params['size'];pred['type']=params['type'];pred['average_time']=params['average_time']
pred['smp_revenue']=pred['smp_price']*30*pred['type']
pred['rec_revenue']=pred['rec_price']*30*pred['type']

,smp_price,rec_price,size,type,average_time,smp_revenue,rec_revenue
date,,,,,,,
2018-04-01,86,100,99,1.2,3.4,3096.0,3600.0
2018-05-01,86,100,99,1.2,3.4,3096.0,3600.0
2018-06-01,85,100,99,1.2,3.4,3060.0,3600.0
2018-07-01,85,100,99,1.2,3.4,3060.0,3600.0
2018-08-01,85,100,99,1.2,3.4,3060.0,3600.0
2018-09-01,84,100,99,1.2,3.4,3024.0,3600.0
2018-10-01,84,100,99,1.2,3.4,3024.0,3600.0
2018-11-01,84,100,99,1.2,3.4,3024.0,3600.0
2018-12-01,83,100,99,1.2,3.4,2988.0,3600.0
